In [38]:
import openpyxl
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.utils import units
from openpyxl.styles import Border, Side
from openpyxl.styles import PatternFill
import requests
from datetime import datetime
import requests
import shutil
import json


In [39]:
def thai_date_format(iso_date_str):
    # แปลง string เป็น datetime object
    dt = datetime.fromisoformat(iso_date_str.replace("Z", "+00:00"))
    
    # สร้างชื่อเดือนภาษาไทย
    months_th = ["", "มกราคม", "กุมภาพันธ์", "มีนาคม", "เมษายน", "พฤษภาคม",
                 "มิถุนายน", "กรกฎาคม", "สิงหาคม", "กันยายน", "ตุลาคม",
                 "พฤศจิกายน", "ธันวาคม"]
    
    day = dt.day
    month = months_th[dt.month]
    year = dt.year + 543  # เปลี่ยนเป็นปี พ.ศ.
    
    return f"{day} {month} {year}"

# ///check Source Form4-1
def check_source_form4_1(ef_source, row, ws_name='ws41'):
    ef_map = {
        'Self collect': 'I',
        'Supplier': 'J',
        'PCR Gen.': 'K',
        'TGO EF': 'L',
        'Int. DB': 'M',
        'Others': 'N'
    }

    if ef_source in ef_map:
        col = ef_map[ef_source]
        cell = f'{col}{row}'
        return f'{ws_name}["{cell}"] = "●"'
    else:
        return f'# ef_source \"{ef_source}\" not recognized'


# กำหนดเส้นกรอบแต่ละด้าน (เส้นบางและเส้นปะ)
thin_border = Border(
    left=Side(border_style="thin", color="000000"),
    right=Side(border_style="thin", color="000000"),
    top=Side(border_style="thin", color="000000"),
    bottom=Side(border_style="thin", color="000000")
)

thin_top = Border(
    top=Side(border_style="thin", color="000000")
)

def set_borderThinTop(ws, cell_range,position):
    rows = ws[cell_range]
    for row in rows:
        for cell in row:
            if position == 'left':
                cell.border = Border(left=Side(style='thin'))
            elif position == 'right':
                cell.border = Border(right=Side(style='thin'))
            elif position == 'top':
                cell.border = Border(
                    top=Side(style='thin'),
                    left=Side(style='thin'),
                    right=Side(style='thin')
                )
            elif position == 'bottom':
                 cell.border = Border(
                    bottom=Side(style='thin'),
                    left=Side(style='thin'),
                    right=Side(style='thin')
                )

    
def set_borderTop(ws, cell_range,position):
    rows = ws[cell_range]
    for row in rows:
        for cell in row:
            if position == 'left':
                cell.border = Border(left=Side(style='dashed'))
            elif position == 'right':
                cell.border = Border(right=Side(style='dashed'))
            elif position == 'top':
                cell.border = Border(top=Side(style='dashed'))
            elif position == 'bottom':
                 cell.border = Border(
                    bottom=Side(style='dashed'),
                    left=Side(style='dashed'),
                    right=Side(style='dashed')
                )
            
def set_borderCenter(ws, cell_range, position):
    rows = ws[cell_range]
    for row in rows:
        for cell in row:
            # อ่าน border เดิมก่อน
            border_old = cell.border

            # กำหนดด้านใหม่ตาม position
            new_border = Border(
                left=border_old.left if position != 'left' else Side(style='mediumDashed'),
                right=border_old.right if position != 'right' else Side(style='mediumDashed'),
                top=border_old.top if position != 'top' else Side(style='mediumDashed'),
                bottom=border_old.bottom if position != 'bottom' else Side(style='mediumDashed')
            )

            cell.border = new_border

def set_border(ws, cell_range):
    rows = ws[cell_range]
    for row in rows:
        for cell in row:
            cell.border = thin_border


In [40]:
form1 = "http://localhost:3000/api/v1/f1/1005/7"

# Fr-01, Fr.03, Fr.03
form1 = requests.get(form1)
if form1.status_code == 200:
    data = form1.json()
    product = data["product"]
    company = data["company"]
    process = data["process"]
else:
    print("เกิดข้อผิดพลาดในการเรียก API:", response.status_code)
start = thai_date_format(product["collect_data_start"])
end = thai_date_format(product["collect_data_end"])
date_range = f"{start} - {end}"
submitted_date_thai = thai_date_format(product.get("submitted_date"))
techinfo_list = json.loads(product["product_techinfo"])
product_techinfo_array = [item.strip() for item in techinfo_list]



In [41]:
form4_1= "http://localhost:3000/api/v1/f4-1/form/7"
form4_1 = requests.get(form4_1)
if form4_1.status_code == 200:
    data41 = form4_1.json()
    form41 = data41["form41"]
    company = data41["company"]
    product = data41["product"]
    process = data41["process"]
    report41Sum = data41["report41Sum"]
else:
    print("เกิดข้อผิดพลาดในการเรียก API:", form4_1.status_code)

In [ ]:
file_path = "../excel/form_CFP.xlsx"
output_path = "../excel/"+company["name"]+"_"+product["product_name_en"]+".xlsx"
shutil.copy(file_path, output_path)

wb = load_workbook(file_path)
ws01 = wb["Fr-01"] # หรือระบุชื่อ sheet: wb["Sheet1"]

ws01["J2"] = date_range
ws01["F5"] = company.get("name", "")
ws01["F6"] = product.get("product_name_th", "")
ws01["J10"] = product.get("product_name_th", "")
ws01["J11"] = product.get("product_name_en", "")
ws01["J12"] = product.get("scope", "")
ws01["J13"] = str(product.get("FU_value", "")) + " " + product.get("FU_th", "")
ws01["J14"] = str(product.get("FU_value", "")) + " " + product.get("FU_en", "")
ws01["J15"] = str(product.get("PU_value", "")) + " " + product.get("PU_th", "")
ws01["J16"] = str(product.get("PU_value", "")) + " " +product.get("PU_en", "")
ws01["J17"] = product.get("sale_ratio", "")

start_row = 19
col = "I"
for idx, info in enumerate(product_techinfo_array):
    cell = f"{col}{start_row + idx}"
    ws01[cell] = info

ws01["J24"] = product.get("pcr_reference", "")
ws01["J25"] = submitted_date_thai

# image 
image_url = "../../"+product.get("product_photo")
img = Image(image_url)  
img.width = 300 
img.height = 300  
ws01.add_image(img, "B12")  
######## ---------------------------------------------------------------------
ws02 = wb["Fr-02"]
for i in range(len(process)):
    input_names = []
    row = 13 + (3 * i)
    row_next = row + 1
    
    # ใส่ชื่อ process
    ws02.merge_cells(f"M{row}:O{row_next}")
    ws02[f"M{row}"] = process[i]['process_name']
    set_border(ws02, f"M{row}:O{row_next}")
    
    # ใส่คำว่า "ขนส่ง" และกำหนดเส้นขอบ
    ws02.merge_cells(f"I{row}:I{row_next}")
    ws02[f"I{row}"] = "ขนส่ง"
    set_border(ws02, f"I{row}:I{row_next}")
    ws02[f"H{row_next}"].border = thin_top
    ws02[f"J{row_next}"].border = thin_top
    ws02[f"K{row_next}"].border = thin_top
    
    
    # รวมชื่อ inputs ที่ไม่ใช่ "ผลิตภัณฑ์"
    input_names = [item['input_name'] for cat in process[i]['inputs'] for item in cat['items']]
    input_names_str = ', '.join(input_names)
    
    # ใส่ข้อมูลรวมชื่อ input
    ws02.merge_cells(f"C{row}:F{row_next}")
    ws02[f"C{row}"] = input_names_str
    set_border(ws02, f"C{row}:F{row_next}")

row_end= row_next + 2
# Box Input
ws02.merge_cells(f"B12:B{row_end-1}")
set_borderTop(ws02, f"B12:B{row_end-1}",'left')
ws02.merge_cells(f"G12:G{row_end-1}")
set_borderTop(ws02, f"G12:G{row_end-1}",'right')
ws02.merge_cells(f"B{row_end}:G{row_end}")
set_borderTop(ws02, f"B{row_end}:G{row_end}",'bottom')
# Box Process
ws02.merge_cells(f"L12:L{row_end-1}")
set_borderTop(ws02, f"L12:L{row_end-1}",'left')
ws02.merge_cells(f"P12:P{row_end-1}")
set_borderTop(ws02, f"P12:P{row_end-1}",'right')
ws02.merge_cells(f"L{row_end}:P{row_end}")
set_borderTop(ws02, f"L{row_end}:P{row_end}",'bottom')
# Box Center
# ws02.merge_cells(f"J8:J{row_end+3}")

set_borderCenter(ws02, f"J8:J{row_end+3}",'right')
    
######## ---------------------------------------------------------------------
ws03 = wb["Fr-03 (2)"]   
fill = PatternFill(start_color='FFFFFF99', end_color='FFFFFF99', fill_type='solid') 
fill_intput_head = PatternFill(start_color='FF92D050', end_color='FF92D050', fill_type='solid')
fill_intput = PatternFill(start_color='FFCCFFCC', end_color='FFCCFFCC', fill_type='solid')
fill_output = PatternFill(start_color='FFFFCC99', end_color='FFFFCC99', fill_type='solid')
fill_waste = PatternFill(start_color='FFCCFFFF', end_color='FFCCFFFF', fill_type='solid')
fill_arrow = PatternFill(start_color='00000000', end_color='00000000', fill_type='solid')
row = 13 
for i in range(len(process)):
    input_names = []
    row_input = row
    row_input_offset = 0
    max_input_rows = 0 

    # ===== Input Section =====
    for j in range(len(process[i]['inputs'])):
        category = process[i]['inputs'][j]
        num_items = len(category['items'])
        current_row = row_input + row_input_offset
        ws03.merge_cells(f"B{current_row}:D{current_row}")
        ws03[f"B{current_row}"] = category['input_cat_name']
        ws03[f"B{current_row}"].fill = fill_intput_head 

        # เขียนรายการ input
        for k in range(num_items):
            ws03[f"B{current_row + k + 1}"] = category['items'][k]['input_name']
            ws03[f"C{current_row + k + 1}"] = category['items'][k]['input_quantity']
            ws03[f"D{current_row + k + 1}"] = category['items'][k]['input_unit']

            ws03[f"B{current_row + k + 1}"].fill = fill_intput
            ws03[f"C{current_row + k + 1}"].fill = fill_intput
            ws03[f"D{current_row + k + 1}"].fill = fill_intput

        row_input_offset += 1 + num_items 
        max_input_rows += 1 + num_items

    # ===== Process Block ด้านขวา =====
    process_row = row
    ws03.merge_cells(f"F{process_row}:I{process_row}")
    ws03[f"F{process_row}"] = process[i]['ordering']
    ws03[f"F{process_row}"].fill = fill
    set_borderThinTop(ws03, f"F{process_row}:I{process_row}", 'top')

    ws03.merge_cells(f"F{process_row + 1}:I{process_row + 5}")
    ws03[f"F{process_row + 1}"] = process[i]['process_name']
    ws03[f"F{process_row + 1}"].fill = fill
    set_borderThinTop(ws03, f"F{process_row + 1}:I{process_row + 5}", 'bottom')

    # ===== output =====
    ws03[f"I{process_row + 7}"] = "ผลิตภัณฑ์"
    ws03[f"I{process_row + 7}"].fill = fill_intput_head 
    for j in range(len(process[i]['outputs'])):
        output = process[i]['outputs'][j]
        ws03[f"I{process_row + 8 + j}"] = output['output_name']
        ws03[f"J{process_row + 8 + j}"] = output['output_quantity']
        ws03[f"K{process_row + 8 + j}"] = output['output_unit']
        ws03[f"I{process_row + 8 + j}"].fill = fill_output
        ws03[f"J{process_row + 8 + j}"].fill = fill_output
        ws03[f"K{process_row + 8 + j}"].fill = fill_output

    # ===== waste =====
    waste_head=["ผลิตภัณฑ์ร่วม", "ของเสีย"]
    row_input_offset = 0
    current_row = row_input + row_input_offset
    waste_total_rows = 0 

    if process[i]['wastes'] == []:
        ws03.merge_cells(f"K{current_row}:M{current_row}")
        ws03[f"K{current_row}"] = waste_head[0]
        ws03[f"K{current_row}"].fill = fill_intput_head
        ws03.merge_cells(f"K{current_row + 1}:M{current_row + 1}")
        ws03[f"K{current_row + 1}"].fill = fill_waste

        ws03.merge_cells(f"K{current_row+2}:M{current_row+2}")
        ws03[f"K{current_row+2}"] = waste_head[1]
        ws03[f"K{current_row+2}"].fill = fill_intput_head
        ws03.merge_cells(f"K{current_row + 3}:M{current_row + 3}")
        ws03[f"K{current_row + 3}"].fill = fill_waste
    else:
        len_waste = len(process[i]['wastes'])
        if(len_waste == 1):
            if process[i]['wastes'][0]['waste_cat_name'] == waste_head[0]:
                ws03.merge_cells(f"K{current_row}:M{current_row}")
                ws03[f"K{current_row}"] = waste_head[0]
                ws03[f"K{current_row}"].fill = fill_intput_head
                for j in range(len(process[i]['wastes'][0]['items'])):
                    item = process[i]['wastes'][0]['items'][j]
                    ws03[f"K{current_row + 2 + j}"] = item['waste_name']
                    ws03[f"L{current_row + 2 + j}"] = item['waste_qty']
                    ws03[f"M{current_row + 2 + j}"] = item['waste_unit']
                    ws03[f"K{current_row + 2 + j}"].fill = fill_waste
                    ws03[f"L{current_row + 2 + j}"].fill = fill_waste
                    ws03[f"M{current_row + 2 + j}"].fill = fill_waste
                
                
                waste_total_rows += len(process[i]['wastes'][0]['items']) + 2

                ws03.merge_cells(f"K{waste_total_rows}:M{waste_total_rows}")
                ws03[f"K{waste_total_rows}"] = waste_head[0]
                ws03[f"K{waste_total_rows}"].fill = fill_intput_head
                ws03.merge_cells(f"K{waste_total_rows+1}:M{waste_total_rows+1}")
                ws03[f"K{waste_total_rows+1}"].fill = fill_waste

            else:

                ws03.merge_cells(f"K{current_row}:M{current_row}")
                ws03[f"K{current_row}"] = waste_head[0]
                ws03[f"K{current_row}"].fill = fill_intput_head
                ws03.merge_cells(f"K{current_row + 1}:M{current_row + 1}")
                ws03[f"K{current_row + 1}"].fill = fill_waste
                ws03.merge_cells(f"K{current_row + 2}:M{current_row + 2}")
                ws03[f"K{current_row + 2}"] = waste_head[1] 
                ws03[f"K{current_row + 2}"].fill = fill_intput_head


                for j in range(len(process[i]['wastes'][0]['items'])):
                    item = process[i]['wastes'][0]['items'][j]
                    ws03[f"K{current_row + 3 + j}"] = item['waste_name']
                    ws03[f"L{current_row + 3 + j}"] = item['waste_qty']
                    ws03[f"M{current_row + 3 + j}"] = item['waste_unit']
                    ws03[f"K{current_row + 3 + j}"].fill = fill_waste
                    ws03[f"L{current_row + 3 + j}"].fill = fill_waste
                    ws03[f"M{current_row + 3 + j}"].fill = fill_waste

                waste_total_rows += len(process[i]['wastes'][0]['items']) + 2

        for j in range(len(process[i]['wastes'])):

            if process[i]['wastes'][0]['waste_cat_name']==waste_head[j]:
                ws03.merge_cells(f"K{current_row}:M{current_row}")
                ws03[f"K{current_row}"] = waste_head[0]
                ws03[f"K{current_row}"].fill = fill_intput_head
                 # เขียนรายการผลิตภัณฑ์ร่วม
                for j in range(len(process[i]['wastes'][0]['items'])):
                    item = process[i]['wastes'][0]['items'][j]
                    ws03[f"K{current_row + 2 + j}"] = item['waste_name']
                    ws03[f"L{current_row + 2 + j}"] = item['waste_qty']
                    ws03[f"M{current_row + 2 + j}"] = item['waste_unit']
                    ws03[f"K{current_row + 2 + j}"].fill = fill_waste
                    ws03[f"L{current_row + 2 + j}"].fill = fill_waste
                    ws03[f"M{current_row + 2 + j}"].fill = fill_waste

                waste_total_rows += len(process[i]['wastes'][0]['items']) + 2
            else:
                ws03.merge_cells(f"K{current_row}:M{current_row}")
                ws03[f"K{current_row}"] = waste_head[0]
                ws03[f"K{current_row}"].fill = fill_intput_head
                ws03.merge_cells(f"K{current_row + 1}:M{current_row + 1}")
            
            ws03[f"K{current_row + 1}"].fill = fill_waste

    # ===== ปรับ row สำหรับ process ถัดไป =====
    ws03[f"E{process_row+2}"].fill = fill_arrow
    ws03[f"J{process_row+2}"].fill = fill_arrow
    # ws03.merge_cells(f"H{process_row+6}:H{process_row+6+4}")
    # ws03[f"H{process_row+6}"].fill = fill_arrow

    if (max_input_rows <=6):
        u=process_row+6
        l=process_row+9
        print("u,l"+ str(u) + "," + str(l))
    else:
        x=max_input_rows-9
        u=process_row+6
        l=process_row+10+x
        print("u,l"+ str(u) + "," + str(l))
    
    if j< len(process)-1:
        ws03.merge_cells(f"G{u}:G{l}")
        ws03[f"G{u}"].fill = fill_arrow

    row += max(max_input_rows + 2, 10) 



######## ---------------------------------------------------------------------
ws41 = wb["Fr-04.1"]

row = 11
phase=["การได้มาของวัตถุดิบ","การผลิต","การกระจายสินค้า","การใช้งาน","การจัดการซาก"]
for i in range(len(phase)):
    for j in range(len(form41[0]["process"])):
        ws41[f"B{row}"] = form41[0]["process"][j]["process_name"]
        for k in range(len(form41[0]["process"][j]["product"])):
            ws41[f"B{row+1}"] = form41[0]["process"][j]["product"][k]
        row += 1
wb.save(output_path)
print("เขียนข้อมูลใน Excel เรียบร้อยแล้ว")

KeyError: 'inputs'

In [35]:
form41[0]["process"][0]["process_name"]

'กระบวนการรับและเตรียมวัตถุดิบ '

In [ ]:
wb.save(output_path)
print("เขียนข้อมูลใน Excel เรียบร้อยแล้ว")

In [ ]:

# ws41 = wb["Fr-04.1"]
# ws41["B11"] = product.get("product_name_th", "")
# ws41["B12"] = form41.get("item_name", "")
# ws41["C12"] = form41.get("item_unit", "")
# ws41["D12"] = form41.get("item_quantity", "")
# ws41["E12"] = float(form41.get("item_quantity", 0) or 0) * 1.3
# ws41["F12"] = form41.get("lci_source_period", "")
# ws41["G12"] = form41.get("ef", "")
# exec(check_source_form4_1(form41.get("ef_source", ""), 12))
# # ws41["H12"] to ["M12"] form41.get("ef_source", "") 
# # ws41["N12"] = subtitle
# ws41["O12"] = form41.get("ef_source_ref", "")
# ws41["P12"] = form41.get("ratio", "")
# ws41["Q12"] = form41.get("ghg_emission", "")
# ws41["R12"] = form41.get("ghg_emission_proportion", "")
# ws41["S12"] = form41.get("cut_off", "")
# ws41["T12"] = form41.get("description", "")